Importing the dependencies


In [7]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

Data Collection and Analysis

In [ ]:
#loading the dataset to pandas dataframe
diabetes_dataset=pd.read_csv('kaggle_diabetes.csv')
diabetes_dataset

In [ ]:
#printing the first five rows of the dataset
diabetes_dataset.head()

In [ ]:
diabetes_dataset.shape

In [ ]:
#statistical measures of the data
diabetes_dataset.describe()

In [ ]:
# Count of null values
diabetes_dataset.isnull().sum()

In [ ]:
diabetes_dataset['Outcome'].value_counts()

In [ ]:
# Outcome countplot
sns.countplot(x = 'Outcome',data = diabetes_dataset)

In [ ]:
#finding out the Blood Pressure and age of entries who have diabetes
plt.figure(figsize=(10,6))
# Scatter with positive example
plt.scatter(diabetes_dataset.Age[diabetes_dataset.Outcome==1],diabetes_dataset.BloodPressure[diabetes_dataset.Outcome==1],c="Red");

# Scatter with negative example
plt.scatter(diabetes_dataset.Age[diabetes_dataset.Outcome==0],diabetes_dataset.BloodPressure[diabetes_dataset.Outcome==0],c="green");

# Add some helpful info
plt.title("Diabetes in function of Age and Blood pressure")
plt.xlabel("Age")
plt.ylabel("Blood Pressure")
plt.legend(["Diabetes","No Diabetes"]);

In [ ]:
# Scatter plot matrix 
from pandas.plotting import scatter_matrix
scatter_matrix(diabetes_dataset, figsize = (20, 20));

In [2]:
# Pairplot 
sns.pairplot(data = diabetes_dataset, hue = 'Outcome')
plt.show()

NameError: name 'sns' is not defined

In [3]:
# Heatmap
sns.heatmap(diabetes_dataset.corr(), annot = True)
plt.figure(figsize=(10,10))
plt.show()

NameError: name 'sns' is not defined

Average of features grouped by Outcome (Diabetes)

In [43]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.168693,110.586626,68.094985,20.052432,70.563830,30.567477,0.434676,31.081307
1,4.732456,141.568713,71.166667,22.633041,98.897661,35.320468,0.540681,36.956140


In [44]:
#separating data and labels #axis = 1 fr dropping a column and 0 for droppinga row
X = diabetes_dataset.drop(columns = 'Outcome', axis=1)
Y = diabetes_dataset['Outcome']


In [45]:
print(X)

      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0               2      138             62             35        0  33.6   
1               0       84             82             31      125  38.2   
2               0      145              0              0        0  44.2   
3               0      135             68             42      250  42.3   
4               1      139             62             41      480  40.7   
...           ...      ...            ...            ...      ...   ...   
1995            2       75             64             24       55  29.7   
1996            8      179             72             42      130  32.7   
1997            6       85             78              0        0  31.2   
1998            0      129            110             46      130  67.1   
1999            2       81             72             15       76  30.1   

      DiabetesPedigreeFunction  Age  
0                        0.127   47  
1                      

In [46]:
print(Y)

0       1
1       0
2       1
3       1
4       0
       ..
1995    0
1996    1
1997    0
1998    1
1999    0
Name: Outcome, Length: 2000, dtype: int64


In [47]:
dataset_new = diabetes_dataset

In [48]:
 #Replacing zero values with NaN
dataset_new[["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]] = dataset_new[["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]].replace(0,np.NaN)

In [49]:
# Count of NaN
dataset_new.isnull().sum()

Pregnancies                   0
Glucose                      13
BloodPressure                90
SkinThickness               573
Insulin                     956
BMI                          28
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [50]:
# Replacing NaN with mean values
dataset_new["Glucose"].fillna(dataset_new["Glucose"].mean(), inplace = True)
dataset_new["BloodPressure"].fillna(dataset_new["BloodPressure"].mean(), inplace = True)
dataset_new["SkinThickness"].fillna(dataset_new["SkinThickness"].mean(), inplace = True)
dataset_new["Insulin"].fillna(dataset_new["Insulin"].mean(), inplace = True)
dataset_new["BMI"].fillna(dataset_new["BMI"].mean(), inplace = True)

In [51]:
# Statistical summary
dataset_new.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,2000.0,3.703500,3.306063,0.000,1.000,3.000000,6.000000,17.00
Glucose,2000.0,121.975340,30.533114,44.000,99.000,118.000000,141.000000,199.00
BloodPressure,2000.0,72.403665,11.949531,24.000,64.000,72.000000,80.000000,122.00
SkinThickness,2000.0,29.341275,9.123652,7.000,25.000,29.341275,32.000000,110.00
Insulin,2000.0,153.743295,80.376307,14.000,120.000,153.743295,153.743295,744.00
BMI,2000.0,32.650101,7.190136,18.200,27.600,32.400000,36.800000,80.60
DiabetesPedigreeFunction,2000.0,0.470930,0.323553,0.078,0.244,0.376000,0.624000,2.42
Age,2000.0,33.090500,11.786423,21.000,24.000,29.000000,40.000000,81.00
Outcome,2000.0,0.342000,0.474498,0.000,0.000,0.000000,1.000000,1.00


***Data Standardization


In [52]:
scaler = StandardScaler()

In [53]:
scaler.fit(X)

StandardScaler()

In [54]:
standardized_data = scaler.transform(X)

In [55]:
print(standardized_data)

[[-0.5153943   0.52455322 -0.37248123 ...  0.17268332 -1.06324616
   1.18042417]
 [-1.12049474 -1.1597562   0.67008046 ...  0.73724853 -0.7355513
  -0.85632626]
 [-1.12049474  0.74288962 -3.60442246 ...  1.47363794  0.49175869
  -0.17740945]
 ...
 [ 0.69480658 -1.12856529  0.46156812 ... -0.12187245 -0.27492362
   0.75610116]
 [-1.12049474  0.24383498  2.12966682 ...  4.28419085 -0.46968566
  -0.60173245]
 [-0.5153943  -1.25332895  0.14879962 ... -0.25687717  0.23516743
  -0.68659705]]


In [56]:
X = standardized_data
Y = diabetes_dataset['Outcome']

In [57]:
print(X)
print(Y)

[[-0.5153943   0.52455322 -0.37248123 ...  0.17268332 -1.06324616
   1.18042417]
 [-1.12049474 -1.1597562   0.67008046 ...  0.73724853 -0.7355513
  -0.85632626]
 [-1.12049474  0.74288962 -3.60442246 ...  1.47363794  0.49175869
  -0.17740945]
 ...
 [ 0.69480658 -1.12856529  0.46156812 ... -0.12187245 -0.27492362
   0.75610116]
 [-1.12049474  0.24383498  2.12966682 ...  4.28419085 -0.46968566
  -0.60173245]
 [-0.5153943  -1.25332895  0.14879962 ... -0.25687717  0.23516743
  -0.68659705]]
0       1
1       0
2       1
3       1
4       0
       ..
1995    0
1996    1
1997    0
1998    1
1999    0
Name: Outcome, Length: 2000, dtype: int64


Train Test Split

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [59]:
print(X.shape, X_train.shape, X_test.shape)

(2000, 8) (1600, 8) (400, 8)


***Training the model

In [65]:
classifier = svm.SVC(kernel='linear')

In [66]:
#training the support vector Machine Classifier
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

Evaluation of Model

In [67]:
#Accuracy Score

In [68]:
#accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [74]:
print('Accuracy score of the training data: ', training_data_accuracy)

Accuracy score of the training data:  0.775


In [70]:
#accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [71]:
print('Accuracy score of the test data: ', test_data_accuracy)

Accuracy score of the test data:  0.805


In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(X_test_prediction, Y_test)
cm

In [75]:
# Classification report
from sklearn.metrics import classification_report
print(classification_report(X_test_prediction, Y_test))

              precision    recall  f1-score   support

           0       0.91      0.81      0.86       295
           1       0.60      0.78      0.68       105

    accuracy                           0.81       400
   macro avg       0.76      0.80      0.77       400
weighted avg       0.83      0.81      0.81       400



Making a predictive system


In [ ]:
input_data = (3,148,66,25,0,32.5,0.256,22)

#changing te input data to anumpy array
input_data_as_numpy_array = np.asarray(input_data)

#reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

#standardizing the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)
    
prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
    print('The person is not diabetic.')
else:
    print('The person is diabetic.')